<a href="https://colab.research.google.com/github/Rachel-0420/Cross-Entropy-Fast-Transient/blob/real-data/cross_entropy_btw_supernovae.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install --upgrade "jax[cuda]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
# !pip install astropy pzflow corner

# from pzflow import Flow
# import jax.numpy as jnp
# import pandas as pd
import numpy as np
import pickle
# import corner
import matplotlib.pyplot as plt
# import os
# from astropy.table import Table
# from pzflow import Flow, FlowEnsemble
# # from pzflow.distributions import Uniform
# from pzflow.bijectors import Chain, StandardScaler, NeuralSplineCoupling, ColorTransform, InvSoftplus, RollingSplineCoupling, ShiftBounds
# from collections import namedtuple

# import scipy.stats as sps

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Paths for the Probability Cubes of some transient classes
# SNII and SNIbc are both supernovae and are similar. AGN is very different from the other two

data1 = "/content/drive/MyDrive/Datacubes/FullRunTest_1000ObjNo/ProbCube_1229_1815__SNII-Templates.pkl"
data2 = "/content/drive/MyDrive/Datacubes/FullRunTest_1000ObjNo/ProbCube_1229_0821__SNIbc-Templates.pkl"
# data3 = "/content/drive/MyDrive/Datacubes/FullRunTest_1000ObjNo/ProbCube_1229_1819__AGN.pkl"

In [4]:
with open(data1, 'rb') as f1:
    CoordinateDict1 = pickle.load(f1)
    Data1 = pickle.load(f1)

with open(data2, 'rb') as f2:
    CoordinateDict2 = pickle.load(f2)
    Data2 = pickle.load(f2)

#with open(data3, 'rb') as f3:
#    CoordinateDict3 = pickle.load(f3)
#    Data3 = pickle.load(f3)

In [5]:
Data2.astype(float)

array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
    

In [6]:
print(Data1.shape)
print(Data2.shape)
shape = Data2.shape

(28, 3603, 110, 38)
(28, 3603, 110, 38)


In [11]:
print(len(CoordinateDict2['BandPairs']))
print(len(CoordinateDict2['dT1s']))
print(len(CoordinateDict2['dT2s']))
print(len(CoordinateDict2['dT1s']) * len(CoordinateDict2['dT2s']))
print(len(CoordinateDict2['BinMag']))
print(len(CoordinateDict2['BinColor']))

28
65
67
4355
111
39


In [7]:
def normalize(data):

  total = 0

  for i in range(shape[0]):
    for j in range(shape[1]):
      for k in range(shape[2]):
        for l in range(shape[3]):
          total += data[i][j][k][l]

  factor = 1 / total

  for i in range(shape[0]):
    for j in range(shape[1]):
      for k in range(shape[2]):
        for l in range(shape[3]):
          data[i][j][k][l] *= factor

  return None

In [8]:
normalize(Data1)
normalize(Data2)

In [ ]:
CoordinateDict1

{'EventName': 'SNII-Templates',
 'PointsPerDay': 1,
 'ObjectNo': 1000,
 'BandPairs': ['ug',
  'ur',
  'ui',
  'uz',
  'gu',
  'gr',
  'gi',
  'gz',
  'gy',
  'ru',
  'rg',
  'ri',
  'rz',
  'ry',
  'iu',
  'ig',
  'ir',
  'iz',
  'iy',
  'zu',
  'zg',
  'zr',
  'zi',
  'zy',
  'yg',
  'yr',
  'yi',
  'yz'],
 'dT1s': array([-480, -465, -450, -435, -420, -405, -390, -375, -360, -345, -330,
        -315, -300, -285, -270, -255, -240, -225, -210, -195, -180, -165,
        -150, -135, -120, -105,  -90,  -75,  -60,  -45,  -30,  -15,    0,
          15,   30,   45,   60,   75,   90,  105,  120,  135,  150,  165,
         180,  195,  210,  225,  240,  255,  270,  285,  300,  315,  330,
         345,  360,  375,  390,  405,  420,  435,  450,  465,  480]),
 'dT2s': array([-1920, -1890, -1860, -1830, -1800, -1770, -1740, -1710, -1680,
        -1650, -1620, -1590, -1560, -1530, -1500, -1470, -1440,  -480,
         -450,  -420,  -390,  -360,  -330,  -300,  -270,  -240,  -210,
         -180,  -150, 

In [ ]:
np.nonzero(Data1)

(array([ 0,  0,  0, ..., 27, 27, 27]),
 array([   0,    0,    0, ..., 3602, 3602, 3602]),
 array([41, 41, 41, ..., 76, 77, 88]),
 array([25, 26, 27, ..., 16, 16, 23]))

In [ ]:
x = np.nonzero(Data2)

In [ ]:
np.where(Data2 != 0)

(array([ 0,  0,  0, ..., 27, 27, 27]),
 array([   0,    0,    0, ..., 3602, 3602, 3602]),
 array([37, 38, 40, ..., 68, 68, 71]),
 array([21, 21, 21, ..., 21, 22, 18]))

In [ ]:
np.shape(Data2)

(28, 3603, 110, 38)

In [ ]:
np.shape(x)

(4, 13741379)

In [ ]:
help(np.nonzero)

Help on _ArrayFunctionDispatcher in module numpy:

nonzero(a)
    Return the indices of the elements that are non-zero.
    
    Returns a tuple of arrays, one for each dimension of `a`,
    containing the indices of the non-zero elements in that
    dimension. The values in `a` are always tested and returned in
    row-major, C-style order.
    
    To group the indices by element, rather than dimension, use `argwhere`,
    which returns a row for each non-zero element.
    
    .. note::
    
       When called on a zero-d array or scalar, ``nonzero(a)`` is treated
       as ``nonzero(atleast_1d(a))``.
    
       .. deprecated:: 1.17.0
    
          Use `atleast_1d` explicitly if this behavior is deliberate.
    
    Parameters
    ----------
    a : array_like
        Input array.
    
    Returns
    -------
    tuple_of_arrays : tuple
        Indices of elements that are non-zero.
    
    See Also
    --------
    flatnonzero :
        Return indices that are non-zero in the fl

In [ ]:
count_nonzero1 = np.sum(Data1 != 0)
count_01 = np.sum(Data1 == 0)
print((count_01, count_nonzero1, count_01 + count_nonzero1))

count_nonzero3 = np.sum(Data3 != 0)
count_03 = np.sum(Data3 == 0)
print((count_03, count_nonzero3, count_03 + count_nonzero3))

(408921098, 12774022, 421695120)
(420485763, 1209357, 421695120)


In [ ]:
x = np.where(Data2 == 0)

In [ ]:
Data2[x] = 1e-16

In [ ]:
print(len(Data2))

28


In [9]:
part1 = Data2[:10]
part2 = Data2[10:20]
part3 = Data2[20:]

In [10]:
part1[np.where(part1 == 0.)] = 1.e-16

In [11]:
part2[np.where(part2 == 0.)] = 1.e-16

In [12]:
part3[np.where(part3 == 0.)] = 1.e-16

In [13]:
for i in range(len(part1)):
  Data2[i] = part1[i]
  Data2[10+i] = part2[i]

for j in range(len(part3)):
  Data2[20+j] = part3[j]

In [14]:
del part1
del part2
del part3

In [15]:
H1 = Data1 * np.log(Data2)

<ipython-input-15-f9dd387c4ab2>:1: RuntimeWarning: divide by zero encountered in log
  H1 = Data1 * np.log(Data2)
<ipython-input-15-f9dd387c4ab2>:1: RuntimeWarning: invalid value encountered in multiply
  H1 = Data1 * np.log(Data2)


In [28]:
del H1
del Data1
del Data2